In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Supongamos que 'data' es el DataFrame cargado con las columnas descritas
data=pd.read_csv('../../Data/Raw/dataset_para_modelas_parte1.csv')
data1=pd.read_csv('../../Data/Raw/dataset_para_modelas_parte2.csv')
data_unida=pd.concat([data,data1]) # Ajusta el nombre del archivo según corresponda
data_unida=data_unida.iloc[:,1:]
data_unida.head()

/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_74952/1687943579.py:10: DtypeWarning: Columns (5,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../../Data/Raw/dataset_para_modelas_parte1.csv')
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_74952/1687943579.py:11: DtypeWarning: Columns (5,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data1=pd.read_csv('../../Data/Raw/dataset_para_modelas_parte1.csv')


,cod_persona,mes,pais,sexo,edad,fecha1,xti_empleado,xti_nuevo_cliente,num_antiguedad,xti_rel,...,ind_prod16,ind_prod17,ind_prod18,ind_prod19,ind_prod20,ind_prod21,ind_prod22,ind_prod23,ind_prod24,ind_prod25
0,178103,2015-01-28,ES,H,35,2015-01-12,N,0.0,6,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
1,503082,2015-01-28,ES,V,27,2012-08-10,N,0.0,35,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
2,502996,2015-01-28,ES,V,37,2012-08-10,N,0.0,35,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
3,503053,2015-01-28,ES,H,23,2012-08-10,N,0.0,35,1.0,...,0,0,0,0,0,0,0.0,0.0,1,0
4,503031,2015-01-28,ES,H,44,2012-08-10,N,0.0,35,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0


In [6]:
# Transformar los datos en una estructura de secuencias temporales

def create_sequences(data, user_id_col, month_col, feature_cols, target_cols, sequence_length=16):
    sequences = []
    targets = []
    user_ids = data[user_id_col].unique()
    for user_id in user_ids:
        user_data = data[data[user_id_col] == user_id].sort_values(by=month_col)
        if len(user_data) >= sequence_length + 1:
            for i in range(len(user_data) - sequence_length):
                seq_features = user_data[feature_cols].iloc[i:i + sequence_length].values
                seq_targets = user_data[target_cols].iloc[i + sequence_length].values
                sequences.append(seq_features)
                targets.append(seq_targets)
    return np.array(sequences), np.array(targets)

In [11]:
categorical_columns = ['pais', 'sexo', 'xti_empleado', 'xti_nuevo_cliente', 'xti_rel', 'xti_rel_1mes', 'tip_rel_1mes',
                       'indresi', 'indext', 'des_canal', 'xti_extra', 'tip_dom', 'cod_provincia', 'id_segmento']
existing_categorical_columns = [col for col in categorical_columns if col in data_unida.columns]
data_unida = pd.get_dummies(data_unida, columns=existing_categorical_columns)
data_unida.head()

,cod_persona,mes,edad,fecha1,num_antiguedad,fec_ult_cli_1t,xti_actividad_cliente,imp_renta,mean_engagement,ind_prod1,...,cod_provincia_46.0,cod_provincia_47.0,cod_provincia_48.0,cod_provincia_49.0,cod_provincia_50.0,cod_provincia_51.0,cod_provincia_52.0,id_segmento_01 - TOP,id_segmento_02 - PARTICULARES,id_segmento_03 - UNIVERSITARIO
0,178103,2015-01-28,35,2015-01-12,6,NaN,1.0,87218.10,6.134600,0,...,0,0,0,0,0,0,0,0,1,0
1,503082,2015-01-28,27,2012-08-10,35,NaN,0.0,70777.59,5.414673,0,...,0,0,0,0,0,0,0,0,0,1
2,502996,2015-01-28,37,2012-08-10,35,NaN,0.0,104035.08,6.301283,0,...,0,0,0,0,0,0,0,0,1,0
3,503053,2015-01-28,23,2012-08-10,35,NaN,1.0,136930.23,5.014352,0,...,0,0,0,0,0,0,0,0,0,1
4,503031,2015-01-28,44,2012-08-10,35,NaN,0.0,110245.53,6.851770,0,...,0,0,0,0,0,0,0,0,1,0


In [12]:
 #Definir las columnas de características y objetivos
feature_cols = ['edad', 'num_antiguedad', 'imp_renta', 'mean_engagement', 'pais', 'sexo', 'xti_empleado', 'xti_nuevo_cliente', 
                'xti_rel', 'xti_rel_1mes', 'tip_rel_1mes', 'indresi', 'indext', 'des_canal', 'xti_extra', 'tip_dom', 
                'cod_provincia', 'xti_actividad_cliente', 'id_segmento']

target_cols = [f'ind_prod{i}' for i in range(1, 26)]

# Crear secuencias y objetivos
sequences, targets = create_sequences(data_unida, 'cod_persona', 'mes', feature_cols, target_cols)

KeyError: "['pais', 'sexo', 'xti_empleado', 'xti_nuevo_cliente', 'xti_rel', 'xti_rel_1mes', 'tip_rel_1mes', 'indresi', 'indext', 'des_canal', 'xti_extra', 'tip_dom', 'cod_provincia', 'id_segmento'] not in index"

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Supongamos que 'data' es el DataFrame cargado con las columnas descritas
data=pd.read_csv('../../Data/Raw/dataset_para_modelas_parte1.csv')
data1=pd.read_csv('../../Data/Raw/dataset_para_modelas_parte1.csv')
data_unida=pd.concat([data,data1])  # Ajusta el nombre del archivo según corresponda



#

# Normalizar las características
scaler = StandardScaler()
num_features = sequences.shape[2]
sequences = sequences.reshape(-1, num_features)
sequences = scaler.fit_transform(sequences)
sequences = sequences.reshape(-1, 16, num_features)

# Dividir los datos
X_train, X_val, y_train, y_val = train_test_split(sequences, targets, test_size=0.2, random_state=42)

# Definir la arquitectura del modelo
model = Sequential([
    LSTM(128, activation='relu', input_shape=(16, num_features), return_sequences=True),
    Dropout(0.5),
    LSTM(64, activation='relu'),
    Dropout(0.5),
    Dense(25, activation='sigmoid')  # 25 salidas para los 25 productos
])

# Compilar el modelo
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Evaluar el modelo
results = model.evaluate(X_val, y_val)
print(f'Validation loss: {results[0]}, Validation accuracy: {results[1]}')

2024-05-17 16:38:31.358794: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_74952/1648794416.py:10: DtypeWarning: Columns (5,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../../Data/Raw/dataset_para_modelas_parte1.csv')
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_74952/1648794416.py:11: DtypeWarning: Columns (5,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data1=pd.read_csv('../../Data/Raw/dataset_para_modelas_parte1.csv')


KeyError: "['pais', 'sexo', 'xti_empleado', 'xti_nuevo_cliente', 'xti_rel', 'xti_rel_1mes', 'tip_rel_1mes', 'indresi', 'indext', 'des_canal', 'xti_extra', 'tip_dom', 'cod_provincia', 'id_segmento'] not in index"